In [1]:
import numpy as np
from gensim.models.keyedvectors import KeyedVectors

import mtruk_loader
from constants import CONSTRACTIONS

# model_file = "./GoogleNews-vectors-negative300.bin.gz"      # 3,000,000 words
# model_file = "./freebase-vectors-skipgram1000-en.bin.gz"    # 1,422,903 words
model_file = "./glove.840B.300d.bin.gz"                     # 2,196,016 words
word2vec = KeyedVectors.load_word2vec_format(model_file, binary=True)

DUMP_300VEC = np.array([0.0 for _ in range(300)])
def Normalization(sentences, max_str_size):
    vec_sentences = []
    seq_length = []

    for sentence in sentences:
        seq = 0

        vec_sentence = []
        for word in sentence.split():
            word = word.replace(".", "").replace(",", "")
            if word in CONSTRACTIONS:
                word = CONSTRACTIONS[word].replace("/", "").replace(";", "")
                for _word in word.split():
                    if _word in word2vec:
                        vec_sentence.append(word2vec.wv[_word])
                        seq += 1

            else:
                if word in word2vec:
                    vec_sentence.append(word2vec.wv[word])
                    seq += 1

        while len(vec_sentence) != max_str_size:
            vec_sentence.append(np.array(DUMP_300VEC))

        vec_sentences.append(vec_sentence)
        seq_length.append(seq)

    return np.array(vec_sentences, dtype=np.float64), seq_length

import tensorflow as tf

MAX_STR = 1000
MAX_VEC = 300
NUM_UNIT = 50
BATCH_SIZE = 30

X1 = tf.placeholder(tf.float64, [None, MAX_STR, MAX_VEC])
X2 = tf.placeholder(tf.float64, [None, MAX_STR, MAX_VEC])
Y = tf.placeholder(tf.float64, [None])

X1_Seq = tf.placeholder(tf.int64, [None])
X2_Seq = tf.placeholder(tf.int64, [None])

batch_size = tf.placeholder(tf.int32, [])

cell = tf.nn.rnn_cell.BasicLSTMCell(NUM_UNIT)

init_state = cell.zero_state(batch_size, tf.float64)

outputs1, states1 = tf.nn.dynamic_rnn(
    cell,
    X1,
    sequence_length=X1_Seq,
    initial_state=init_state,
    dtype=tf.float64
)

outputs2, states2 = tf.nn.dynamic_rnn(
    cell,
    X2,
    sequence_length=X2_Seq,
    initial_state=init_state,
    dtype=tf.float64
)


outputs1 = outputs1[:, -1, :]
outputs2 = outputs2[:, -1, :]

logits = tf.exp(-tf.norm(outputs1-outputs2, axis=1))


In [9]:

loss = tf.reduce_mean(tf.square(Y - logits))
train = tf.train.AdadeltaOptimizer(learning_rate=0.001).minimize(loss)





In [14]:

data_file = "./merged_mtruk.csv"
train, val, test = mtruk_loader.load_data(data_file)

BATCH_RANGE = train.batch_size // BATCH_SIZE

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver = tf.train.Saver()

    x_size = 100
    for step in range(1000):

        _loss = 0
        for batch_step in range(BATCH_RANGE):
            sentence_a, sentence_b, answer = train.next_batch(BATCH_SIZE, one_hot=False)

            train_a, seq_a = Normalization(sentence_a, MAX_STR)
            train_b, seq_b = Normalization(sentence_b, MAX_STR)

            train_feed_dict = {
                X1: train_a,
                X2: train_b,
                X1_Seq: seq_a,
                X2_Seq: seq_b,
                Y: answer/5,
                batch_size: len(seq_a)
            }
            
            l, _ = sess.run([loss, train], feed_dict=train_feed_dict)
            _loss += l;
            print(l)

            print("%04d - %03d :" % (step, BATCH_RANGE), batch_step)
            # sess.run(cell.zero_state(batch_size[0], tf.float64), feed_dict={batch_size: [len(test1_sequence)]})

        if step % 200 == 0:
            saver.save(sess, "lstm_saver/lstm", global_step=step)


TypeError: Fetch argument <mtruk_loader.DataSet object at 0x7f4b94f03e80> has invalid type <class 'mtruk_loader.DataSet'>, must be a string or Tensor. (Can not convert a DataSet into a Tensor or Operation.)